In [3]:
from pymer4.models import Lmer
# For data manipulation and analysis
import pandas as pd
# For statistics and econometrics
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
# For numerical operations
import numpy as np
# For visualization
import matplotlib.pyplot as plt
import seaborn as sns


### Linear mixed effect model using pymer4 or statsmodels

In [10]:


# Reload the data
df = pd.read_excel('Rew#1_Coef_filtered_new.xlsx')
run_to_time = {'r01': 0, 'r02': 1, 'r03': 2}
df['time'] = df['run'].map(run_to_time)
# df['group'] = pd.Categorical(df['group'], categories=['no_stim', 'DLPFC_cTBS', 'M1_cTBS'], ordered=True)


# Define the ROIs
rois_left = ['laPU', 'laCA', 'lpPU', 'lpCA']
rois_right = ['raPU', 'raCA', 'rpPU', 'rpCA']
rois_PU = ['laPU','raPU','lpPU','rpPU']
rois_CA = ['laCA','raCA','lpCA','rpCA']


# Convert 'run' to numerical values representing the order of time

# Recode the 'group' variable to use 'no_stim' as the baseline level
# data_all_groups['group_recoded'] = pd.Categorical(data_all_groups['group'], 
#                                                   categories=['no_stim', 'DLPFC_cTBS', 'M1_cTBS'], 
#                                                   orde_red=True)


# Define the regions as anterior and posterior
anterior_regions = ['laPU', 'laCA', 'raPU', 'raCA']
posterior_regions = ['lpPU', 'lpCA', 'rpPU', 'rpCA']

# Create a new column 'region' in the data for anterior vs posterior comparision
df["region"] = "anterior"
df.loc[df['roi'].isin(posterior_regions), "region"] = "posterior"

## Make a label for right and left ROIs
df["LR"] = "left"
df.loc[df['roi'].isin(rois_right), "LR"] = "right"

# Define PC for comparing putamen versus caudate nucleus
df["PC"] = "PU"
df.loc[df['roi'].isin(rois_CA), "PC"] = "CA"
# Convert 'region' to categorical
df["region"] = pd.Categorical(df["region"])


# Perform the analysis for each ROI
interaction_results_time_recoded = {}
mixedlm_result = {}
for roi in rois_left + rois_right:
    roi_data = df[df['roi'] == roi].dropna()
    
    # Fit the mixed effects model
#     mixedlm_result = smf.mixedlm("beta ~ time * group", roi_data, groups=roi_data['subj']).fit()
    model = Lmer('beta ~ time * group + (1|subj)',data =roi_data )
# model = smf.mixedlm("beta ~ time * group_recoded", roi_data, groups=roi_data['subj'], re_formula="~time")
    mixedlm_result[roi] = model.fit()
    
    # Get the p-value of the interaction term
    print(mixedlm_result[roi])
#     interaction_results_time_recoded[roi] = {f"time:group_recoded[T.{group}]": p_val for group, p_val in mixedlm_result.pvalues.items() if "group_recoded" in group}

#     print(mixedlm_result.summary())


Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*group+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 192	 Groups: {'subj': 64.0}

Log-likelihood: -44.934 	 AIC: 105.868

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.068  0.260
Residual               0.049  0.221

No random effect correlations specified

Fixed effects:

                      Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  \
(Intercept)              0.601   0.484    0.719  0.060   97.567  10.002   
time                    -0.133  -0.189   -0.077  0.029  125.000  -4.663   
groupDLPFC_cTBS         -0.024  -0.220    0.172  0.100   97.567  -0.244   
groupM1_cTBS             0.005  -0.191    0.201  0.100   97.567   0.048   
time:groupDLPFC_cTBS    -0.083  -0.176    0.010  0.047  125.000  -1.752   
time:groupM1_cTBS        0.052  -0.041    0.145  0.047  125.000   1.103   

                      P-val  Sig  
(Intercept)           0.000  ***  
time     

### ANOVA among group difference for the initial fMRI activity (~10 minute)

In [11]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Subset the data to include only the first run
for roi in rois_left + rois_right:
    df_first_run = df[(df['run'] == 'r01') & (df['roi'] == roi)]  # replace 'r01' with the correct value for the first run in your data

    # Fit the model
    model = ols('beta ~ group', data=df_first_run).fit()

    # Perform the ANOVA
    anova_table = sm.stats.anova_lm(model, typ=2)

    print(anova_table)


            sum_sq    df         F    PR(>F)
group     0.007092   2.0  0.044343  0.956657
Residual  4.878059  61.0       NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.255806   2.0  1.216573  0.303329
Residual  6.413166  61.0       NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.034784   2.0  0.323626  0.724755
Residual  3.278172  61.0       NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.070531   2.0  0.339899  0.713182
Residual  6.328969  61.0       NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.045067   2.0  0.334716  0.716847
Residual  4.106603  61.0       NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.026174   2.0  0.147608  0.863077
Residual  5.408274  61.0       NaN       NaN
            sum_sq    df        F    PR(>F)
group     0.068982   2.0  0.72749  0.487264
Residual  2.892069  61.0      NaN       NaN
            sum_sq    df         F    PR(>F)
group     0.1

In [12]:
df['group'].unique()

array(['baseline', 'M1_cTBS', 'DLPFC_cTBS'], dtype=object)

### For each stimulation group, analyze the time dependency

In [13]:
for group in df['group'].unique():
    model = Lmer('beta ~ time + (1|subj)', data = df[df['group'] == group])
    print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 720	 Groups: {'subj': 30.0}

Log-likelihood: -105.248 	 AIC: 218.496

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.064  0.254
Residual               0.068  0.260

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  P-val  Sig
(Intercept)     0.426   0.331    0.522  0.049   32.766   8.736    0.0  ***
time           -0.114  -0.137   -0.090  0.012  689.000  -9.568    0.0  ***
Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 408	 Groups: {'subj': 17.0}

Log-likelihood: 2.523 	 AIC: 2.954

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.055  0.235
Residual               0.049  0.222

No random effect correlations specified

Fixed effects:



### For each stimulation group and ROI, analyze the time dependency


In [156]:
for group in df['group'].unique():
    for roi in rois_left + rois_right:
        model = Lmer('beta ~ time + (1|subj)', data = df[(df['group'] == group) & (df['roi'] == roi)])
        print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 90	 Groups: {'subj': 30.0}

Log-likelihood: -29.812 	 AIC: 67.624

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.087  0.295
Residual               0.060  0.245

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE      DF  T-stat  P-val  Sig
(Intercept)     0.601   0.469    0.734  0.068  45.827   8.893    0.0  ***
time           -0.133  -0.195   -0.071  0.032  59.000  -4.198    0.0  ***
Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 90	 Groups: {'subj': 30.0}

Log-likelihood: -22.649 	 AIC: 53.298

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.060  0.246
Residual               0.055  0.234

No random effect correlations specified

Fixed effects:

    

### More analyses

In [166]:
df_CA = df[df['roi'].str.endswith('CA')]
df_PU = df[df['roi'].str.endswith('PU')]
df_DLPFC_cTBS = df[df['group']=='DLPFC_cTBS']
df_baseline = df[df['group']=='no_stim']
df_M1_cTBS = df[df['group']=='M1_cTBS']
df_DLPFC_cTBS_CA = df[(df['roi'].str.endswith('CA')) & (df['group']=='DLPFC_cTBS')]
df_DLPFC_cTBS_PU = df[(df['roi'].str.endswith('PU')) & (df['group']=='DLPFC_cTBS')]
df_DLPFC_cTBS_LCA = df[(df['roi'].str.endswith('CA')) & (df['group']=='DLPFC_cTBS') & (df['LR']=='left')]
df_DLPFC_cTBS_Ant = df[(df['group']=='DLPFC_cTBS') & (df['region']=='anterior')]


In [167]:
model = Lmer('beta ~ time * PC + (1|subj)',data = df_DLPFC_cTBS_Ant)
print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*PC+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 204	 Groups: {'subj': 17.0}

Log-likelihood: -3.133 	 AIC: 18.266

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.040  0.200
Residual               0.045  0.213

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  P-val  Sig
(Intercept)     0.520   0.404    0.635  0.059   28.626   8.815  0.000  ***
time           -0.254  -0.305   -0.203  0.026  184.000  -9.828  0.000  ***
PCPU            0.076  -0.016    0.169  0.047  184.000   1.615  0.108     
time:PCPU       0.046  -0.025    0.118  0.037  184.000   1.268  0.206     


In [168]:
model = Lmer('beta ~ time * region + (1|subj)',data =df_DLPFC_cTBS)
print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*region+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 408	 Groups: {'subj': 17.0}

Log-likelihood: 2.423 	 AIC: 7.153

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.037  0.192
Residual               0.049  0.221

No random effect correlations specified

Fixed effects:

                      Estimate  2.5_ci  97.5_ci     SE      DF  T-stat  P-val  \
(Intercept)              0.558   0.455    0.661  0.053   23.41  10.627  0.000   
time                    -0.231  -0.268   -0.194  0.019  388.00 -12.172  0.000   
regionposterior         -0.213  -0.281   -0.146  0.035  388.00  -6.164  0.000   
time:regionposterior     0.051  -0.001    0.104  0.027  388.00   1.911  0.057   

                      Sig  
(Intercept)           ***  
time                  ***  
regionposterior       ***  
time:regionposterior    .  


In [102]:
model = Lmer('beta ~ time * region * group + (1|subj)',data =df)
print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*region*group+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 1536	 Groups: {'subj': 64.0}

Log-likelihood: 5.916 	 AIC: 16.168

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.055  0.235
Residual               0.049  0.221

No random effect correlations specified

Fixed effects:

                                   Estimate  2.5_ci  97.5_ci     SE        DF  \
(Intercept)                           0.558   0.437    0.679  0.062    79.516   
time                                 -0.231  -0.268   -0.194  0.019  1463.000   
regionposterior                      -0.213  -0.281   -0.146  0.035  1463.000   
groupM1_cTBS                         -0.021  -0.193    0.151  0.088    79.516   
groupno_stim                         -0.033  -0.185    0.119  0.078    79.516   
time:regionposterior                  0.051  -0.001    0.104  0.027  1463.000   
time:groupM1_cTBS                

In [101]:
model = Lmer('beta ~ time * region * group + (1|subj)',data =df_PU )
print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*region*group+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 768	 Groups: {'subj': 64.0}

Log-likelihood: 60.656 	 AIC: -93.311

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.047  0.217
Residual               0.037  0.192

No random effect correlations specified

Fixed effects:

                                   Estimate  2.5_ci  97.5_ci     SE       DF  \
(Intercept)                           0.596   0.477    0.715  0.061   94.097   
time                                 -0.208  -0.253   -0.162  0.023  695.000   
regionposterior                      -0.313  -0.396   -0.229  0.043  695.000   
groupM1_cTBS                         -0.002  -0.170    0.167  0.086   94.097   
groupno_stim                          0.003  -0.146    0.152  0.076   94.097   
time:regionposterior                  0.108   0.043    0.173  0.033  695.000   
time:groupM1_cTBS                     0

In [98]:
model = Lmer('beta ~ time * region * group + (1|subj)',data =df_CA )
print(model.fit())

Linear mixed model fit by REML [’lmerMod’]
Formula: beta~time*region*group+(1|subj)

Family: gaussian	 Inference: parametric

Number of observations: 768	 Groups: {'subj': 64.0}

Log-likelihood: -65.145 	 AIC: 158.289

Random effects:

                 Name    Var    Std
subj      (Intercept)  0.067  0.260
Residual               0.052  0.227

No random effect correlations specified

Fixed effects:

                                   Estimate  2.5_ci  97.5_ci     SE       DF  \
(Intercept)                           0.520   0.378    0.662  0.072   93.259   
time                                 -0.254  -0.308   -0.200  0.028  695.000   
regionposterior                      -0.114  -0.213   -0.016  0.050  695.000   
groupM1_cTBS                         -0.041  -0.241    0.160  0.102   93.259   
groupno_stim                         -0.068  -0.245    0.109  0.090   93.259   
time:regionposterior                 -0.005  -0.082    0.071  0.039  695.000   
time:groupM1_cTBS                     

In [16]:
# model = smf.mixedlm("beta ~ time * group", data_all_groups, groups=data_all_groups["subj"])
model = smf.mixedlm("beta ~ group * time", df, groups=df["subj"])

result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
==================================================================
Model:                 MixedLM    Dependent Variable:    beta     
No. Observations:      1536       Method:                REML     
No. Groups:            64         Scale:                 0.0598   
Min. group size:       24         Log-Likelihood:        -129.5079
Max. group size:       24         Converged:             Yes      
Mean group size:       24.0                                       
------------------------------------------------------------------
                       Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               0.451    0.060   7.541 0.000  0.334  0.569
group[T.M1_cTBS]       -0.037    0.085  -0.442 0.658 -0.203  0.128
group[T.baseline]      -0.025    0.075  -0.331 0.741 -0.172  0.122
time                   -0.205    0.015 -13.847 0.000 -0.234 -0.176
group[T.M1_cTBS]:time   0.134    0.021   6.383 0.000  0.093  0.175
group[T.baseline]:time  0.092    0.019   4.932 0.000  0.055  0.128
Group Var               0.055    0.043                            
==================================================================

"""